ETP, exchange traded products, track underlying securities, indexes, or other financial instruments, similarly to stock these are traded on exchanges which implies daily price fluctuations.Each ETP can be include as many underelying assets, digital ones also, as the creator decides, thus, gaining and losing in value in sync with the underlying assets, ETFs are the msot popular types of ETPs. These financial basket type of instruments are a great cost-effective way for institutions and individuals to gain exposure to a specific market, therefore crypto ETPs are great for investors willing to gain an exposure to crypto without the hassle of holding coins on spot or wallets.

The following is an EDA, exploratory data analysis, aiming for insights about:

ETP's liquidity (bid/ask spreads & volume)
Underlying assets' liquidity (bid/ask spreads & volume)
Relevant correlations

In [1]:
import pandas as pd
import pandas_datareader as web
import numpy as np
from datetime import datetime
import seaborn as sns

from matplotlib import pyplot as plt
import matplotlib as matplot
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import requests
import json
from pandas.plotting import register_matplotlib_converters

#ETP csv file downloaded from six-group website to extract a list of ETP names & ISIN (Internation Security Identification Numbers)
df = pd.read_csv('ETP Explorer export_splittedintocolumns.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   FundLongName                125 non-null    object 
 1   ValorSymbol                 125 non-null    object 
 2   FundReutersTicker           125 non-null    object 
 3   FundBloombergTicker         125 non-null    object 
 4   ValorNumber                 125 non-null    int64  
 5   ISIN                        125 non-null    object 
 6   IssuerLongNameDesc          125 non-null    object 
 7   IssuerNameFull              125 non-null    object 
 8   TradingBaseCurrency         125 non-null    object 
 9   FundCurrency                125 non-null    object 
 10  ManagementFee               125 non-null    float64
 11  ReplicationMethodDesc       125 non-null    object 
 12  ManagementStyleDesc         125 non-null    object 
 13  MarketMakers                125 non

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,...,DailyLowPrice,OnMarketVolume,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,BTCB.S,BTCB SE,113027698,CH1130276988,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,ETHB.S,ETHB SE,113027699,CH1130276996,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,EUR,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,USD,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In order to have a list of inputs that is recognizable by Yahoo Finance API, we need to add a .SW to each line in the VALORSYMBOL column.

In [2]:
df.head()

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,...,DailyLowPrice,OnMarketVolume,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,BTCB.S,BTCB SE,113027698,CH1130276988,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,ETHB.S,ETHB SE,113027699,CH1130276996,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,EUR,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,USD,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Creating a column of default value .SW to add to ValorSymbol column values/line to obtain YahooTicker for our data collection.

df['.SW'] = '.SW'
df['YahooTicker'] = df['ValorSymbol'] + df['.SW']
df.head()

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,...,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc,.SW,YahooTicker
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,BTCB.S,BTCB SE,113027698,CH1130276988,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.SW,BTCB.SW
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,ETHB.S,ETHB SE,113027699,CH1130276996,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.SW,ETHB.SW
2,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.SW,BTCA.SW
3,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,EUR,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.SW,BTCA.SW
4,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,USD,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.SW,BTCA.SW


Checking the columns' names to know what to drop and keep.

In [4]:
#Let's look at a preview of our data
print(df.columns)

Index(['FundLongName', 'ValorSymbol', 'FundReutersTicker',
       'FundBloombergTicker', 'ValorNumber', 'ISIN', 'IssuerLongNameDesc',
       'IssuerNameFull', 'TradingBaseCurrency', 'FundCurrency',
       'ManagementFee', 'ReplicationMethodDesc', 'ManagementStyleDesc',
       'MarketMakers', 'ClosingPrice', 'ClosingPerformance', 'ClosingDelta',
       'FundUnderlyingDescription', 'BidVolume', 'BidPrice', 'AskVolume',
       'AskPrice', 'MidSpread', 'PreviousClosingPrice', 'MarketDate',
       'MarketTime', 'OpeningPrice', 'OpeningPrice.1', 'DailyLowPrice',
       'OnMarketVolume', 'OffBookVolume', 'TotalTurnover', 'TotalTurnoverCHF',
       'ProductLineDesc', 'AssetClassDesc', 'UnderlyingGeographicalDesc',
       'LegalStructureCountryDesc', 'UnderlyingProviderDesc', '.SW',
       'YahooTicker'],
      dtype='object')


If we fetch the data from yahooFinance, we care about the columns 'YahooTicker', 'FundLongName', 'ISIN', 'FundCurrency's,'FundUnderlyingDecription'

In [5]:
#Dropping unnecessary columns

df.drop(columns = [ 'ValorSymbol', 'FundReutersTicker',
       'FundBloombergTicker', 'ValorNumber', 'IssuerLongNameDesc',
       'IssuerNameFull', 'TradingBaseCurrency', 
       'ManagementFee', 'ReplicationMethodDesc', 'ManagementStyleDesc',
       'MarketMakers', 'ClosingPrice', 'ClosingPerformance', 'ClosingDelta',
       'BidVolume', 'BidPrice', 'AskVolume',
       'AskPrice', 'MidSpread', 'PreviousClosingPrice', 'MarketDate',
       'MarketTime', 'OpeningPrice', 'OpeningPrice.1', 'DailyLowPrice',
       'OnMarketVolume', 'OffBookVolume', 'TotalTurnover', 'TotalTurnoverCHF',
       'ProductLineDesc', 'AssetClassDesc', 'UnderlyingGeographicalDesc',
       'LegalStructureCountryDesc', 'UnderlyingProviderDesc'], axis = 1, inplace = True)

#Option to view all rows to check for errors in YahooTicker column
pd.set_option('display.max_rows', None)
df

,FundLongName,ISIN,FundCurrency,FundUnderlyingDescription,.SW,YahooTicker
0,1 FiCAS Active Bitcoin Exchange Traded Product,CH1130276988,CHF,Bitcoin (BTC),.SW,BTCB.SW
1,1 FiCAS Active Ethereum Exchange Traded Product,CH1130276996,CHF,Ethereum (ETH),.SW,ETHB.SW
2,15 FiCAS Active Crypto ETP,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
3,15 FiCAS Active Crypto ETP,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
4,15 FiCAS Active Crypto ETP,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
5,21Shares Algorand (ALGO) ETP,CH1146882316,USD,NaN,.SW,ALGO.SW
6,21Shares Algorand (ALGO) ETP,CH1146882316,USD,NaN,.SW,ALGO.SW
7,21Shares Algorand (ALGO) ETP,CH1146882316,USD,NaN,.SW,ALGO.SW
8,21Shares Avalanche (AVAX) ETP,CH1135202088,USD,NaN,.SW,AVAX.SW
9,21Shares Avalanche (AVAX) ETP,CH1135202088,USD,NaN,.SW,AVAX.SW


We will rely on a dictionary with YahooTicker as key values for data fetching from Yahoo Finance.
And we will also collect BTC data to add to our ETP database for analysis purpose.

Dictionary Creation, YahooTicker in keys & FundLongName in values.

In [6]:
#Creating our dicrtionary then assigning FundLongName and YahooTicker to a dictionary
#First assigning each column to a list 

ETP_dict = dict(zip(df.YahooTicker, df.FundLongName))
ETP_dict.pop('ALGO.SW')
ETP_dict.pop('AVAX.SW')

#Uncomment below line to preview of our dictionary
print(ETP_dict)

{'BTCB.SW': '1 FiCAS Active Bitcoin Exchange Traded Product', 'ETHB.SW': '1 FiCAS Active Ethereum Exchange Traded Product', 'BTCA.SW': '15 FiCAS Active Crypto ETP', 'ABNB.SW': '21Shares Binance BNB ETP', 'ABCH.SW': '21Shares Bitcoin Cash ETP', 'ABTC.SW': '21Shares Bitcoin ETP', 'ABBA.SW': '21Shares Bitcoin Suisse Index ETP', 'KEYS.SW': '21Shares Bitwise Select 10 Large Cap Crypto Index ETP', 'AADA.SW': '21Shares Cardano ETP', 'ATOM.SW': '21Shares Cosmos ETP (ATOM)', 'HODLX.SW': '21Shares Crypto Basket 10 ETP', 'HODLV.SW': '21Shares Crypto Basket Equal Weight ETP', 'HODL.SW': '21Shares Crypto Basket Index', 'ALTS.SW': '21Shares Crypto Mid-Cap Index ETP (ALTS)', 'AETH.SW': '21Shares Ethereum ETP', 'ADOT.SW': '21Shares Polkadot (ADOT) ETP', 'POLY.SW': '21Shares Polygon (POLY) ETP', 'AXRP.SW': '21Shares Ripple XRP ETP', 'SBTC.SW': '21Shares Short Bitcoin ETP', 'ASOL.SW': '21Shares Solana (SOL) ETP', 'AXLM.SW': '21Shares Stellar ETP', 'LUNA.SW': '21Shares Terra ETP (LUNA)', 'AXTZ.SW': '21Sh

In [7]:
#Collecting BTC daily data from yahoo finance
import yfinance as yf

start_date = "2020-01-01"

#Setting today's date
today_date = datetime.today().strftime("%Y-%m-%d")
btc_df = yf.download("BTC-USD", start_date, end_date = "2022-01-01")
btc_df.drop(columns= ['Adj Close'], axis = 1, inplace = True)
btc_df.rename(columns= {"Open":"BTC-Open", "Close":"BTC-Close", "Volume" : "BTC-Volume"}, inplace = True)

delta = btc_df['BTC-Open'] - btc_df['BTC-Close']
btc_df['Delta'] = delta 
btc_df.head()

[*********************100%***********************]  1 of 1 completed


,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,-5.282227
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465,217.081055
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,-360.455566
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275,-65.281250
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095,-0.865723


In [8]:
#Given the random nature of variations between OPEN & CLOSE we expect some negative Delta Hr Trend values 
#Verification of negative values existences
(btc_df < 0).any().any()

True

ETP_df is our final dataframe, including BTC data + ETPs data.

In [9]:
ETP_df = btc_df.copy()

ETP_dict
col_list = []
for key, value in ETP_dict.items():
    print(key, value)
    try:
        df_temp = web.DataReader(
            key, start_date, end_date = "2020-12-01", data_source = "yahoo"
        )
    except ValueError:
        print(f'{key} symbol not found')
    df_temp.index = pd.to_datetime(df_temp.index)
    df_temp.rename(columns = {"Close": value}, inplace = True)
    ETP_df = pd.merge(
        left = ETP_df,
        right = df_temp[value],
        how = 'inner',
        left_index = True, right_index = True)
ETP_df.head()



BTCB.SW 1 FiCAS Active Bitcoin Exchange Traded Product


TypeError: DataReader() got an unexpected keyword argument 'end_date'